<a href="https://colab.research.google.com/github/JoDeMiro/DeepLearningIntroduction/blob/main/10_Reinforcement_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Forrás:

https://thebojda.medium.com/tensorflow-alapoz%C3%B3-4-cfeee8b9e34c


# Reinforcement Learning

Annak idején bejárta a tech sajtót a hír, hogy a Google által felvásárolt DeepMind olyan mesterséges intelligenciát fejlesztett, ami pusztán a képernyő figyelésével képes volt megtanulni Atari játékokat játszani.

Annak ellenére, hogy a dolog baromi izgalmas, azért felmerül az emberben a kérdés, hogy miért vásárol fel a Google fél milliárd dollárért egy céget, ahol retro játékokhoz készítenek mesterséges intelligenciát? Nos, egy rendszer, ami képes megtanulni retro játékokat játszani, ugyanúgy képes lehet elvezérelni egy robotot is ami mondjuk homokzsákokat pakol árvíz idején, vagy éppen embereket ment ki egy földrengés helyszínéről.

A DeepMind által használt technikát Deep Q-learningnek nevezik, és a reinforcement learning (megerősítéses tanulás) egy formája.


<img src="https://miro.medium.com/max/300/1*6KEByEw2ZlEhe8EIVgXFlw.png" />



<img src="https://miro.medium.com/max/702/1*4u2GtNnMa9xso1WkLh7hVA.png" />



 Minél tovább játszik, annál több adata lesz arról, hogy melyik akció milyen jutalommal jár, így annál pontosabban tudja megtippelni a Q értékét.

 Akármi is legyen a konkrét feladat, minden esetben ráhúzható a fenti képen látható modell. Minden esetben van valamilyen környezet (environment), amiben a robotunk (agent) létezik. Ez lehet egy számítógépes játék egyszerű szabályokkal, de lehet akár a valós tér is (aminek működését a fizika szabályai határozzák meg). Ezt a környezetet értelmezi a robot. A környezet absztrakt értelmezése az állapot (state). A robot képes műveleteket (action) végezni, amik hatással vannak a környezetre. Egy számítógépes játék esetén a műveletek száma elég korlátozott (egy Atari játéknál pl. egy joystick mozgatása fel/le/jobbra/balra), míg egy valós robot esetén sokkal többféle mozgás elképzelhető. A művelet elvégzését követően a környezet valahogy változik, amiből újra generálódik az absztrakt állapot (state) és még egy fontos dolog: a jutalom (reward). A jutalom miatt hívjuk ezt a tanulási módszert megerősítéses tanulásnak, hiszen ahogyan az állatok idomítása esetén, itt is a sikeres akciókat jutalmazzuk, a sikerteleneket pedig büntetjük. A robot célja, hogy minél jobban elvégezze a feladatát, tehát minél nagyobb értékben gyűjtsön össze reward-ot (pl. egy számítógépes játék esetén minél több pontot szerezzen).

 A DeepMind által használt Deep Q-learning lényege, hogy az aktuális állapot (state) alapján minden egyes művelethez (action) megpróbáljuk megtippelni a összes várható jutalmat (reward), és mindig azt a műveletet választjuk, amelyik a legnagyobb jutalommal kecsegtet. A Q-learningben a Q (quality of action) ezt az elvárható jutalmat jelenti. A játék indulásakor még semmit nem tud a rendszer a Q értékéről, így csak vaktában próbálkozik felderíteni a játékot.

```
#  Minél tovább játszik, annál több adata lesz arról, hogy melyik akció milyen
jutalommal jár, így annál pontosabban tudja megtippelni a Q értékét.
```

Fontos megjegyezni, hogy egy akcióhoz tartozó Q nem az akcióért közvetlenül kapható jutalom értékét jelenti, hanem azt, hogy ha az adott akciót választva végigjátsszuk a játékot, mekkora összes jutalomra számíthatunk.

Vegyünk például egy labirintusos játékot, ahol a cél kijutni a labirintusból úgy, hogy eközben a legtöbb kincset szedjük fel. Választanunk kell, hogy jobbra, vagy balra megyünk. Balra van egy kincs, viszont arra nem lehet kijutni a labirintusból. Jobbra nincs semmi, de ha a következő sarkon jobbra fordulunk, akkor kijutunk a labirintusból. Ha csak közvetlenül az akció által elérhető rewardot néznénk, minden esetben balra kellene mennünk, hiszen arra van a kincs (nagyobb reward), míg a jobbra menéssel közvetlenül nem nyerünk semmit (0 reward). Ennek fényében a Q értékét a következőképpen számoljuk ki:

$$Q(s) = max_a(R(s,a) + \gamma Q(s'))$$

A fenti képletet Bellman egyenletnek hívjuk és valójában nem túl bonyolult. Az s jelenti az állapotot, az a az akciót. Az R(s,a) az adott állapotból adott akcióval elérhető jutalom (reward), a Q(s) pedig az adott állapotból elérhető legnagyobb összes jutalom. Hogy megkapjuk a Q értékét, az aktuális akcióval elérhető jutalomhoz (R(s,a)) adjuk hozzá az akció által előállt következő állapot (s’) várható Q értékét megszorozva egy gamma konstanssal, ami azt jelképezi, hogy a jövőben várható jutalom mennyit ér nekünk a jelenben. Tehát fogjuk az akcióval nyerhető jutalmat, hozzáadjuk a jövőben elvárható jutalom “diszkontált” értékét, és azt az akciót választjuk, ahol ez a legnagyobb. Dióhéjban igazából ennyi a Q-learning lényege. Attól lesz belőle Deep Q-learning, hogy a Q érték megtippelésére neurális hálót használunk.

### OpenAI Gym

Mielőtt elkezdenénk kódolni, ismerkedjünk meg az OpenAI Gymmel, ami pont az, amire elsőre gondol az ember: egy edzőterem robotoknak. Kicsit pontosabban: a Gym tulajdonképpen egy szabványos interfész bármilyen környezethez. A programkönyvtár letöltésével eleve kapunk egy csomó környezetet (pl. Atari játékokat), de igény esetén további szimulációs környezetekkel is bővíthető. Vannak olyan környezetek, amelyek komoly fizikai engine-el rendelkeznek, így akár valós robotok szimulált környezetben való tanítására is megfelelőek lehetnek (inkább itt törjünk össze pár robotot a tanítás során, mint a valóságban). Lássuk hogy néz ki mindez a gyakorlatban.



In [1]:
import gym

def query_environment(name):
  env = gym.make(name)
  spec = gym.spec(name)
  print(f"Action Space: {env.action_space}")
  print(f"Observation Space: {env.observation_space}")
  print(f"Max Episode Steps: {spec.max_episode_steps}")
  print(f"Nondeterministic: {spec.nondeterministic}")
  print(f"Reward Range: {env.reward_range}")
  print(f"Reward Threshold: {spec.reward_threshold}")

/usr/local/lib/python3.7/dist-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [2]:
query_environment("MountainCar-v0")

Action Space: Discrete(3)
Observation Space: Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
Max Episode Steps: 200
Nondeterministic: False
Reward Range: (-inf, inf)
Reward Threshold: -110.0


In [3]:
query_environment("CartPole-v1")

Action Space: Discrete(2)
Observation Space: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
Max Episode Steps: 500
Nondeterministic: False
Reward Range: (-inf, inf)
Reward Threshold: 475.0


## Render OpenAI Gym Environments from CoLab

It is possible to visualize the game your agent is playing, even on CoLab. This section provides information on how to generate a video in CoLab that shows you an episode of the game your agent is playing. This video process is based on suggestions found <a href="https://colab.research.google.com/drive/1flu31ulJlgiRL1dnN2ir8wGh9p7Zij2t">here</a>.

Begin by installing **pyvirtualdisplay** and python-opengl.

In [4]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

Next, we install needed requirements to display an Atari game.

In [5]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Next we define functions used to show the video by adding it to the CoLab notebook.

In [2]:
import gym
from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment 
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

Now we are ready to play the game. We use a simple random agent.

In [13]:
#env = wrap_env(gym.make("MountainCar-v0"))
env = wrap_env(gym.make("Atlantis-v0"))

observation = env.reset()

while True:
  
    # env.render()
    env.render(mode='rgb_array')
    
    #your agent goes here
    action = env.action_space.sample() 
         
    observation, reward, done, info = env.step(action) 
   
        
    if done: 
      break;
            
env.close()
show_video()

## Na tehát annyit biztosan tudunk, hogy

Orbitális cumi Google Colab alatt futtatni az OpenAI Gym modult.

## Visszatérve az eredeti projekthez

Ha simán így futtatjuk, ahogy az a nagykönyvben meg van írva, kurvaélet, hogy nem fogunk semmit látni.

In [12]:
import gym

env = gym.make("Breakout-v0")

observation = env.reset()
for _ in range(1000):
    # env.render()
    env.render(mode='rgb_array')
    action = env.action_space.sample()  # your agent here (this takes random actions)
    observation, reward, done, info = env.step(action)

    if done:
        observation = env.reset()
env.close()

## Na de,

Azért vagyunk programozók, hogy kiheckeljünk mindent. Mivel fent már megirtam a show_vide() függvényt ezt most is bevetjük.

In [11]:
import gym

# env = gym.make("Breakout-v0")
# helyett
env = wrap_env(gym.make("Breakout-v0"))


observation = env.reset()
for _ in range(1000):
    # env.render()
    env.render(mode='rgb_array')
    action = env.action_space.sample()  # your agent here (this takes random actions)
    observation, reward, done, info = env.step(action)

    if done:
        observation = env.reset()
env.close()
show_video()

A fenti nagyon egyszerű kód a Breakout nevű Atari játék környezetet tölti be, majd véletlenszerűen játszik 10 000 fordulót. Sok értelme nincs, viszont szépen láthatjuk, hogyan működik a Gym. A 3. sorban inicializáljuk a környezetet. Itt adjuk meg, hogy a Breakout játékkal szeretnénk játszani. Az elérhető környezetek <a href="https://gym.openai.com/envs/">listája itt található</a>. A 4. sorban alapállapotba hozzuk a környezetet a reset függvénnyel. A függvény visszatérési értéke egy observation tenzor, ami egy pillanatkép a környezet aktuális állapotáról. Ez lehet valami előemésztett dolog, pl. a labda helyzete és sebességvektora, de lehet simán egy képernyőkép is, mint a DeepMind esetén. Hogy pontosan milyen az observation tenzor szerkezete, az a környezettől függ. Az env.render() megjeleníti a környezet aktuális állapotát, így szemmel is követhetjük a történéseket. Az env.action_space változóban kérhetjük el a környezetben végezhető akciókat, a sample függvény pedig rögtön választ is egyet közölük véletlenszerűen, megspórolva nekünk pár sor kódot. Az env.step függvény alkalmazza az akciót a környezetre, amitől az meg fog változni. A függvény visszatérési értéke a megváltozott környezetről egy új pillanatkép (observation), az akcióért kapott közvetlen jutalom (reward), egy boolean változó, ami azt jelzi, hogy véget ért a játék (done), és némi kiegészítő infó (info). Az utolsó előtti pár sorban ellenőrizzük a done változó értékét, és ha véget ért a játék, alap állapotba hozzuk a környezetet (reset). Ennyi igazából elég is a Gym-ről. Jöhet az igazi kódolás.

## Cart Pole

A kedvencem.

A Gym-ben elérhető egyik legegyszerűbb játék a CartPole, ahol egy kis virtuális kocsit kell vezérelnünk, ami egy botot egyensúlyoz. Az ehhez tartozó Gym-es animáció elég sután néz ki, így inkább nézzünk egy videót arról, ahogyan egy igazi robot csinálja ugyanezt a betanított algoritmus segítségével.

<a href="https://youtu.be/0wSAGwgzrp8">Youtube</a>

In [23]:
from IPython.display import HTML
HTML("""
<iframe width="560" height="315" src="https://www.youtube.com/embed/0wSAGwgzrp8"
frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen>
</iframe>
""")

Szóval a cél a fenti robot betanítása, hogy minél tovább képes legyen egyensúlyozni a bottal. Az alábbi kód az én megvalósításom, amit pár másik kódból lapátoltam össze. A kód elérhető Google Colabban is, így aki akarja, rögtön ki is próbálhatja.

In [3]:
# Deep Q-learning CartPole example
# based on https://github.com/gsurma/cartpole.git & https://gym.openai.com/evaluations/eval_OeUSZwUcR2qSAqMmOE1UIw/

import gym
import random
import numpy as np
from os import path
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

GAMMA = 0.99
LEARNING_RATE = 0.001

MEMORY_SIZE = 1000
BATCH_SIZE = 64

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.1
EXPLORATION_DECAY = 0.99

# env = gym.make("CartPole-v0")
# helyett, hogy a Google Colabban is látni lehessen
env = wrap_env(gym.make("CartPole-v0"))

observation_space_size = env.observation_space.shape[0]
action_space_size = env.action_space.n

model = Sequential()
model.add(Dense(16, input_shape=(observation_space_size, ), activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(action_space_size, activation='linear'))
model.summary()

#if path.exists("cartpole_weights.index"):
#    print("Loading weights from cartpole_weights...")
#    model.load_weights("cartpole_weights")

model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))

exploration_rate = EXPLORATION_MAX
memory = deque(maxlen=MEMORY_SIZE)
run = 0

# while True:
for _ in range(100):
    run += 1
    observation = env.reset()
    step = 0
    while True:
        step += 1
        # env.render()
        # env.render(mode='rgb_array')

        if np.random.rand() < exploration_rate:
            action = random.randrange(action_space_size)
        else:
            state = np.reshape(observation, [1, observation_space_size])
            q_values = model.predict(state)
            action = np.argmax(q_values[0])

        observation_next, reward, done, info = env.step(action)
        reward = reward if not done else -200
        memory.append((observation, action, reward, observation_next, done))
        observation = observation_next

        if done:
            print("Run: " + str(run) + ", exploration: " + str(exploration_rate) + ", score: " + str(step))
            
            if len(memory) >= BATCH_SIZE:
                state_batch, qvalue_batch = [], []
                batch = random.sample(memory, BATCH_SIZE)
                for observation, action, reward, observation_next, done in batch:
                    q_update = reward
                    if not done:
                        state_next = np.reshape(observation_next, [1, observation_space_size])
                        q_predicted = np.amax(model.predict(state_next)[0])
                        q_update = reward + (GAMMA * q_predicted)
                    state = np.reshape(observation, [1, observation_space_size])
                    q_values = model.predict(state)
                    q_values[0][action] = q_update
                    state_batch.append(state[0])
                    qvalue_batch.append(q_values[0])
                model.fit(np.array(state_batch), np.array(qvalue_batch), batch_size=len(state_batch), epochs=1, verbose=0)
                #model.save_weights("cartpole_weights")
        
                exploration_rate *= EXPLORATION_DECAY
                exploration_rate = max(EXPLORATION_MIN, exploration_rate)

            break

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                80        
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 2)                 34        
                                                                 
Total params: 386
Trainable params: 386
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Run: 1, exploration: 1.0, score: 13
Run: 2, exploration: 1.0, score: 44
Run: 3, exploration: 1.0, score: 16
Run: 4, exploration: 0.99, score: 22
Run: 5, exploration: 0.9801, score: 12
Run: 6, exploration: 0.9702989999999999, score: 37
Run: 7, exploration: 0.96059601, score: 11
Run: 8, exploration: 0.9509900498999999, score: 54
Run: 9, exploration: 0.9414801494009999, score: 21
Run: 10, exploration: 0.9320653479069899, score: 14
Run: 11, exploration: 0.92274469442792, score: 13
Run: 12, exploration: 0.9135172474836407, score: 40
Run: 13, exploration: 0.9043820750088043, score: 17
Run: 14, exploration: 0.8953382542587163, score: 19
Run: 15, exploration: 0.8863848717161291, score: 18
Run: 16, exploration: 0.8775210229989678, score: 15
Run: 17, exploration: 0.8687458127689781, score: 21
Run: 18, exploration: 0.8600583546412883, score: 35
Run: 19, exploration: 0.8514577710948754, score: 14
Run: 20, exploration: 0.8429431933839266, score: 13
Run: 21, exploration: 0.8345137614500874, score: 1

In [5]:
show_video()

In [4]:
model.save_weights("cartpole_weights")

A kód elején inicializálunk pár konstanst a Q-learninghez, majd inicializáljuk a Gym-et és lekérdezzük az observation valamint az action tenzor méretét. A játék tenzorainak részletes leírása itt található a játék forráskódjával együtt. Az observation tenzor a CartPole esetén egy 4 elemű vektor. Az első elem az autó helyzetét adja meg, a második az autó sebességét, a harmadik elem a bot dőlésszöge, míg a negyedik a bot dőlésének sebessége. Ez alapján kell nekünk két akció közül választani. Az egyik akció a kocsi jobbra mozgatása, a másik a balra mozgatás. A játék akkor ér véget, ha a bot dőlésszöge 12 foknál nagyobb, vagy sikerült 200 lépésen keresztül egyensúlyozni a botot. A tanításhoz a következő egyszerű hálózatot fogjuk használni:

```
model = Sequential()
model.add(Dense(16, input_shape=(observation_space_size, ), activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(action_space_size, activation='linear'))
model.summary()
```

A hálózat 3 rétegből áll. Az első teljesen huzalozott réteg 4 bemenettel rendelkezik (ekkora az aktuális állapotot leíró observation tenzor mérete), amit 16 kimenetre képez le. A következő réteg egy 16 bemenettel és 16 kimenettel rendelkező teljesen huzalozott réteg, míg az utolsó réteg a 16 bemenetet képzi le 2 kimenetre, ami az adott akciókhoz (jobbra/balra mozgás) tartozó Q érték. Ez összesen 386 állítható paramétert fog jelenteni.

A modell optimalizálásához a szokásos Adam-et fogjuk használni, hibafüggvénynek pedig a négyzetes hibák átlagát (mean squared error), mivel a konkrét Q értékek betanítása a cél.
Maga a program 2 egymásba ágyazott ciklusból áll. A külső ciklusban futtatjuk a játékokat, a belső ciklus pedig egy játék végigjátszása. Ennek megfelelőn a belső ciklus mindig akkor ér véget, ha az env.step metódus játék végét jelző done visszaadott értéke igaz.

A belső ciklus elején található kódblokk határozza meg a következő akciót.

```
if np.random.rand() < exploration_rate: 
  action = random.randrange(action_space_size) 
else: 
  state = np.reshape(observation, [1, observation_space_size])
  q_values = model.predict(state)
  action = np.argmax(q_values[0])
```

Itt kezdjünk az else ággal, amiben a hálózat által tippelt Q érték alapján megyünk tovább. Az első sor megfelelő formába hozza az observation vektort, a neurális háló ugyanis [batch, input] formában várja a bemeneteket. A predict függvénnyel futtatjuk a hálózatot, majd az argmax-al meghatározzuk a nagyobb Q értékhez tartozó akciót.

Mivel kezdetben semmit nem tudunk a környezetről, ezért véletlenszerűen választunk a lehetséges lépések között. Ehhez bevezetünk egy exploration_rate változót, ami azt mondja meg, hogy a következő lépésben milyen valószínűséggel választunk véletlen akciót (felfedezünk) ahelyett, hogy a tippelt Q értéknek megfelelően mennénk tovább. Az exploration_rate-et minden lépésben csökkentjük, így minél okosabb a hálózat, annál inkább a tippelt Q értékekre hagyatkozunk a véletlen választás helyett. Érdemes definiálni egy minimális exploration_rate értéket, hogy mindig legyen véletlen próbálkozás, ezzel elkerülve hogy lokális optimumokban elakadjon az algoritmus. Felmerülhet a kérdés, hogy ha úgyis véletlen súlyokkal inicializáljuk a hálózatot, miért nem elég simán a tippelt Q értékre támaszkodni, ami a futások elején valamilyen véletlen függvénye az állapotnak? Ezzel az a baj, hogy ahogy tanul a rendszer hamar megtanulja, hogy merre vannak reward-ok, és mindig afelé menne. Emlékezzünk vissza a labirintusos példára. Ha balra van egy kincs, jobbra viszont nincs semmi, viszont jobbra van a kijárat, akkor egy hálózat ami rátalált a kincsre, soha nem indulna meg jobbra, így a rendszer elakadna egy lokális optimumban. Azzal, hogy bevezetjük az exploration_rate-et és a véletlen lépéseket, “ki tudjuk rázni” a rendszert az ilyen lokális optimumokból.

```
observation_next, reward, done, info = env.step(action)
reward = reward if not done else -200
memory.append((observation, action, reward, observation_next, done))
observation = observation_next
```

A középső blokkban alkalmazzuk az akciót a környezetre, majd egy memóriában letároljuk az adatokat, vagyis azt, hogy milyen állapotból milyen akcióval milyen állapotba jutottunk el. Ez később a tanulásnál lesz érdekes.

Az utolsó blokk a tanulás, ami az egyes játékok végén történik, és ami tulajdonképpen az egész rendszer esszenciája.

```
state_batch, qvalue_batch = [], []
batch = random.sample(memory, BATCH_SIZE)
for observation, action, reward, observation_next, done in batch:
  q_update = reward
  if not done:
    state_next = np.reshape(observation_next, [1, observation_space_size])
    q_predicted = np.amax(model.predict(state_next)[0])
    q_update = reward + (GAMMA * q_predicted)
  state = np.reshape(observation, [1, observation_space_size])
  q_values = model.predict(state)
  q_values[0][action] = q_update
  state_batch.append(state[0])
  qvalue_batch.append(q_values[0])
model.fit(np.array(state_batch), np.array(qvalue_batch), batch_size=len(state_batch), epochs=1, verbose=0)
```

A kód elején véletlenszerűen kiválasztunk pár mintát a memóriából (ezek mennyisége a BATCH_SIZE konstansban van definiálva). Ezekkel a mintákkal fogjuk az adott fordulóban tanítani a hálózatot. A q_update mezőbe kerül a Q érték, ami végállapot esetén megegyezik a reward-al, más esetben pedig a Bellman egyenletnek megfelelően számoljuk. Ez utóbbi esetben tehát fogjuk a rewardot, majd hozzáadjuk a modellünk által tippelt Q érték GAMMA szorosát. A következő pár sorban lefuttatjuk a hálózatunkat a memóriában szereplő állapotra, majd a memóriában szereplő akcióhoz tartozó Q értéket frissítjük a most számolttal. A state-eket és az akciókhoz számolt Q értékeket egy batch-be pakoljuk, majd egyben betanítjuk a hálózatnak. Lényegében ennyi a Q-learning. Minden körben választunk pár mintát a memóriából, a memória alapján újraszámoljuk a Q értékeket, majd betanítjuk a hálózatnak, egészen addig, míg a hálózat nem lesz elég pontos. A fenti kódot Colabban futtatva nekem olyan 7000 iteráció után állt be konstansan a maximális 200 pontra, de addig is szépen látszik az emelkedés a számokban (amik azt jelentik, hogy hány lépésen keresztül sikerült egyensúlyozni a botot).

Sokan csalódottak lehetnek, hisz a cikk elején Atari játékokat játszó MI-ről írtam, erre kaptak egy vacak botot egyensúlyozó robotot. A helyzet azonban az, hogy valójában Atari játékokat játszani sem sokkal bonyolultabb. Akit érdekel a konkrét megvalósítás, az <a href="https://github.com/gsurma/atari">itt megtalálja a forráskódot</a>. Gyorsan szaladjunk is végig a főbb részeken és különbségeken.

Nyilván az egyik különbség maga a hálózat, amit a <a href="https://github.com/gsurma/atari/blob/master/convolutional_neural_network.py">convolutional_neural_network.py</a> állományban találunk. Igazából nem sokban különbözik a mi botos megoldásunktól. A mi megoldásunk egy 3 rétegű teljesen huzalozott hálózat, aminek a bemenete a kocsi és a bot helyzete valamint sebessége. Mivel az Atari játékoknál a bemenet egy pixeles kép, ezért a helyzetet és sebesség adatokat is a hálózatnak kell kinyernie. Ehhez a fenti példában (és a DeepMind megoldásában is) egy 3 rétegű konvolúciós hálót használnak. Tehát ez a hálózat mindössze abban különbözik a mi hálózatunktól, hogy az elejére raktak pár konvolúciós réteget a szükséges adatok kinyerésére, amit a cartpole-os játék esetén mi készen kaptunk.

ersze a dolog azért nem ennyire egyszerű. Ha megnézünk egy képkockát a Breakout játékból (pl. azt ami a cikk elején található), rájöhetünk, hogy ebből nem fogjuk megtudni sem a labda mozgásának irányát, sem annak sebességét. Ahhoz, hogy ezeket meghatározhassuk, több képkockára van szükség. A Gym-es Breakout esetén az observation tenzor 250x160x3-as méretű és olyan szerkezetű, mint amit a cikksorozat első részében a CIFAR10-es képek kódolására használtunk. Ekkora képre nincs szükség (kisebb monitoron is lehet Breakout-ot játszani), ráadásul a színeknek sincs jelentősége (fekete-fehér monitoron is lehet Breakout-ot játszani). Éppen ezért első körben a képet 84x84x1-es szürke árnyalatos tenzorokká konvertálják, majd ezekből összerakják az aktuális és az azt megelőző három fázis képkockáit. Végül az így kapott 84x84x4-es tenzorok képzik majd a konvolúciós háló bemenetét, amiből már kinyerhető minden szükséges adat. Ezt a fenti program a <a href="https://github.com/gsurma/atari/blob/master/gym_wrappers.py">gym_wrappers.py</a>-ben található csomagoló osztállyal oldja meg, aminek eredményeként a gym környezet által visszaadott observation tenzor már a kívánt 84x84x4-es formában fog jönni.

Van egy harmadik trükk is. A tanítás folyamán a program két ugyanolyan szerkezetű hálózatot használ. Egyet a Q értékek tippeléséhez (target network) és egyet a tanításhoz. A tanítás során módosult hálózat alapján néha frissítjük a target networkot is, de nem minden körben. Ezzel a trükkel hatékonyabbá tehető a tanítás, amit a <a href="https://github.com/gsurma/atari/blob/master/game_models/ddqn_game_model.py">ddqn_game_model.py</a>-ban található DDQNSolver osztály valósít meg.

Igazából ez a 3 főbb dolog az, amiben a fenti (univerzális!) Atari játékokat játszani képes program a mi primitív bot egyensúlyozó robotunktól küölnbözik.

Körülbelül ennyit szerettem volna írni a Q-learningről és a megerősítéses tanulásról. Fontos megjegyezni, hogy a deep reinfocement learning egy igen forró terület. Naponta jönnek ki publikációk, új topológiák és új algoritmusok. Némelyikük a Q-learning valamilyen módosított változata, míg mások ettől eltérő módon működnek. Azt gondolom, hogy a fenti írás jó kiinduló alap lehet bárkinek. Akit pedig mélyebben érdekel a téma, az az Interneten rengeteg anyagot találhat a továbblépéshez.

UPDATE: Tensorflow-hoz készült egy <a href="https://www.tensorflow.org/agents">Tensorflow Agents</a> nevű programkönyvtár, ami reinforcement learninghez tartalmaz mindenféle okosságot valamint az ismertebb módszerek implementációját, így akit mélyebben érdekel a téma, mindenképpen látogasson el a projekt oldalára.

## Nem biztos, hogy ezek kellenek a Colab-hoz

De a biztonság kedvéért most még itt hagyom.

In [ ]:
!python -m pip install pyvirtualdisplay
!pip3 install box2d
!sudo apt-get install xvfb

In [ ]:
%%capture
!pip install -U gym>=0.21.0
!pip install -U gym[atari,accept-rom-license]

In [ ]:
import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar
!mkdir rars
!mv HC\ ROMS.zip   rars
!mv ROMS.zip  rars
!python -m atari_py.import_roms rars